### Variational Autoencoder

In [4]:
### importing modules that are needed

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn 
import torch.nn.functional as f 
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split



##### **Loading the data**

In [5]:
data_f = pd.read_csv(\
    '~/ml_J1-J2_supervised/all_phase/af/augumented_dataL24.csv',index_col=[0])

X_train,X_test,y_train,y_test = train_test_split(data_f.iloc[:,:-1],data_f.iloc[:,-1:], \
                    random_state=42,test_size=0.2,stratify=data_f.iloc[:,-1:])

data_train = pd.concat([X_train,y_train],axis=1)
data_test = pd.concat([X_test,y_test],axis=1)



**Set the device**

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##### Class definition that is used to load the dataset.

In [12]:
### class to load the data

class LoadData(Dataset):
    def __init__(self,data,L,device=device):
        self.L = L
        self.x_data = torch.tensor(data.iloc[:,:-1].values,dtype=torch.float32).to(device=device)
        self.y_data = torch.tensor(data.iloc[:,-1:].values,dtype=torch.long).to(device=device)

    ### length of the dataset
    ### function one has to use if you want to define a custom dataset class
    def __len__(self):
        return len(self.y_data)

    
    ## get the image and label
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.to_list()
        
        image = self.x_data[idx,:]
        label = self.y_data[idx]

        return {'data':image,'label':label}

##### **Model definition for the variational autoencoder**

In [52]:
### definition of the network
class encoder(nn.Module):
    
    def __init__(self,input_size,latent_dim):
        super(encoder,self).__init__()
        self.latent_dim = latent_dim
        self.input_size = input_size
        ### encoder network
        self.encoder1 = nn.Linear(in_features=self.input_size,out_features=256)
        self.encoder2 = nn.Linear(in_features=256,out_features=128)
        self.encoder3 = nn.Linear(in_features=128,out_features=64)
        self.encoder4 = nn.Linear(in_features=64,out_features=32)

        self.mu = nn.Linear(in_features=32,out_features=self.latent_dim)
        self.var = nn.Linear(in_features=32,out_features=self.latent_dim)

    ### feedforward network 
    def forward(self,x):
        ##  passing the input through the network
        x = self.encoder1(x)
        x = self.encoder2(x)
        x = self.encoder3(x)
        x = self.encoder4(x)

        ## passing input x to layer mu and var, mux = g(x), sigmax = f(x) 
        x1 = self.mu(x)
        x2 = self.var(x)

        ## combining mu and sigma to a normal distribution reparameterization trick
        ##  z = mu + sigma * N(0,1)
        #zi = torch.distributions.Normal(0,1).rsample([self.latent_dim])
        #x = x1 + x2*zi

        return x,x1,x2



### definition of the network
class decoder(nn.Module):

    def __init__(self, input_size, latent_dim):
        super(decoder, self).__init__()
        self.latent_dim = latent_dim
        self.input_size = input_size
       
        ### decoder network
        self.decoder1 = nn.Linear(in_features=self.latent_dim, out_features=32)
        self.decoder2 = nn.Linear(in_features=32, out_features=64)
        self.decoder3 = nn.Linear(in_features=64, out_features=128)
        self.decoder4 = nn.Linear(in_features=128, out_features=self.input_size)

    ### feedforward network
    def forward(self, x):
        ##  passing the input through the network
       
        x = self.decoder1(x)
        x = self.decoder2(x)
        x = self.decoder3(x)
        x = self.decoder4(x)

        return x


### Steps of the process
* Pass the input $X$ through the encoder stage $x_enc$
* To get estimate of $\mu$ and $\sigma$ use two neural network and pass $x_enc$ through both of them
    * $\mu_{x} = f_{2}(f_{1}(x))$
    * $\sigma_{x} = g_{2}(f_{1}(x))$
    * Network $f_{1}(x)$ represents the input that is passed through various stages (neural network) and $f_{2},g_{2}$ represents two different neural network.
* To generate the distribution $q(z|x)$ use $\mu_{x},\sigma_{x}$ to generate a normal distribution.
    * $q(z|x) = \mathcal{N}(\mu_{x},\sigma_{x})$
    * Sample a point from this distribution $q(z|x)$
* Now using $z$ we have to generate $p(x|z)$.

In [56]:
input_size = 576
latent_dim = 32
x = torch.randn(input_size)

### create an instance of encoder network
encdr = encoder(input_size,latent_dim)

## generate encoded output and mu,sigma values
enc,mu,sigma = encdr(x)

## create an instance of decoder network
decdr = decoder(input_size,latent_dim)

## generate distribution
q  = torch.distributions.Normal(mu,sigma).rsample()
x_new = decdr(x)




tensor([-3.5051e-02, -5.5179e-05,  1.8747e-01,  8.6430e-02, -3.6078e-02,
         8.7773e-02,  6.3990e-02, -4.3571e-02,  3.2093e-02, -9.9308e-02,
        -1.3944e-01,  2.8436e-01,  7.1158e-02, -1.3529e-01, -1.4323e-01,
        -2.4819e-01, -1.6410e-01,  1.1253e-01,  3.3471e-02, -2.1864e-01,
        -1.2842e-01, -1.4076e-01,  7.2643e-02,  5.4157e-02,  6.4891e-02,
         3.2861e-01, -6.6576e-02, -1.6196e-01,  1.7775e-01, -2.3813e-02,
        -3.5401e-01, -2.0571e-01], grad_fn=<AddBackward0>)
tensor([ 0.0509, -0.0699, -0.0345,  0.1407,  0.0108, -0.2321,  0.0914, -0.2044,
        -0.0070,  0.0246, -0.0931,  0.0832, -0.0717,  0.0971,  0.2505, -0.0178,
         0.0095,  0.1251, -0.0062,  0.0823, -0.0024, -0.2768,  0.0653,  0.2125,
        -0.2291, -0.1535,  0.0096,  0.1226, -0.0060, -0.1472, -0.0033, -0.0599],
       grad_fn=<AddBackward0>)
tensor([ 0.0309,  0.0352,  0.1227, -0.0903, -0.0117,  0.0471, -0.1803,  0.0660,
         0.0904, -0.2177,  0.1588, -0.1339,  0.2165,  0.1844,  0.0851, 